# EDA 5: Pokročilejší manipulace a statistika

* Práce s indexy
* Segmentace, agregace, groupby, pivot
* ~~Elementární optimalizace (%timeit)~~
* ~~Pokročilejší vizualizace se seaborn~~
* ~~Základy statistických testů~~
* Data: IMDB+...?


Nějaká motivace:
* https://github.com/alanderex/pydata-pandas-workshop/tree/master/notebooks
* https://github.com/TomAugspurger/pandas-head-to-tail/blob/master/notebooks/03-Iterators-Groupby.ipynb
* https://realpython.com/pandas-groupby/

In [1]:
import pandas as pd
%matplotlib inline

## Analýza dat po skupinách

Jedním ze základních postupů v datové analýze je rozdělení dat do skupin, aplikace nějaké operace na jednotlivé skupiny a nakonec kombinace výsledků do vhodného datasetu. Anglicky se tento postup označuje jako *split-apply-combine*. Skupiny jsou často, byť ne nutně, definovány nějakou (kategorickou) proměnnou, např. by to mohla být barva, pohlaví nebo kontinent. Skupiny lze ale vytvářet i odvozováním, např. pomocí rozsahu nebo nebo vlastností časových řad. Oboje už jsme vlastně viděli v EDA 3, kdy jsme skupiny vytvářeli pro deštivé dny nebo jednotlivé roky.

Pojďme si to vysvětlit prakticky. Použijeme k tomu hezký dataset se údaji z amerického kongresu.

In [4]:
# odkaz přímo na csv soubor na internetu
LEGISLATORS_HISTORICAL_URL_CSV = (
    "https://theunitedstates.io/congress-legislators/legislators-historical.csv"
)

In [5]:
# některé sloupce uložíme rovnou jako kategorické
dtypes = {
    "gender": "category",
    "type": "category",
    "state": "category",
    "party": "category",
}

legislators_historical = pd.read_csv(
    LEGISLATORS_HISTORICAL_URL_CSV,
    dtype=dtypes,
    usecols=list(dtypes) + ["birthday", "first_name", "last_name"],
    parse_dates=["birthday"],
)

In [6]:
legislators_historical

,last_name,first_name,birthday,gender,type,state,party
0,Bassett,Richard,1745-04-02,M,sen,DE,Anti-Administration
1,Bland,Theodorick,1742-03-21,M,rep,VA,NaN
2,Burke,Aedanus,1743-06-16,M,rep,SC,NaN
3,Carroll,Daniel,1730-07-22,M,rep,MD,NaN
4,Clymer,George,1739-03-16,M,rep,PA,NaN
...,...,...,...,...,...,...,...
11976,Collins,Chris,1950-05-20,M,rep,NY,Republican
11977,Cummings,Elijah,1951-01-18,M,rep,MD,Democrat
11978,Hill,Katie,1987-08-25,F,rep,CA,Democrat
11979,Isakson,John,1944-12-28,M,sen,GA,Republican


### Krok 1: Split

Na rozdělení dat do skupin slouží metoda [`groupby`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html?highlight=groupby#pandas.DataFrame.groupby). Nejjednodušší a možná i nejčastější použití je seskupení podle existujícího sloupce v tabulce. Třeba podle `type`, tedy u nás konkrétně podle komory: `rep` je sněmovna reprezentatntů, `sen` je senát.

In [7]:
legislators_historical_by_type = legislators_historical.groupby("type")
legislators_historical_by_type

Dostali jsme objekt typu [`GroupBy`](https://pandas.pydata.org/pandas-docs/stable/reference/groupby.html). Pokud vám to připomíná `DatetimeIndexResampler` z [Explorativní analýzy a statistiky jedné proměnné](https://naucse.python.cz/2020/pydata-praha-jaro/pydata/eda-univariate-timeseries/), je to velice dobrá asociace. Tato operace totiž data jen rozdělila do skupin, ještě jsme ale neřekli co s těmi skupinami chceme dělat.

### Krok 2 + 3: Apply + Combine

Na skupiny, které jsme vytvořili pomocí `groupby`, teď můžeme aplikovat nějakou operaci. To je právě krok *apply*. Nemá pro nás moc cenu oddělovat tento krok od třetího kroku *combine*, protože když už nějakou opraci aplikujeme, tak bychom rádi výsledek sestavili do výsledného datasetu. Pandas navíc tyto kroky sám nijak neodděluje.

*Pozn.: Apply a combine začne být více odděleno v nástrojích na zpracování velkých dat, které už se nevejdou pohodlně do operační paměti počítače a pandas na ně už nestačí. Apply pak probíhá po částech, třeba i distribuovaně na oddělených serverech, a výsledné combine se provádí sesbíráním částečných výsledků.* 

Použijeme teď jednoduchou agregační metodu `count`, která nám vrátí počet hodnot (po skupinách samozřejmě).

In [8]:
legislators_historical_by_type.count()

,last_name,first_name,birthday,gender,state,party
type,,,,,,
rep,10151,10151,9658,10151,10151,9925
sen,1830,1830,1770,1830,1830,1822


Vidíme, že v datech je zaznamenáno 1830 senátorů a 10151 kongresmanů. Na levé straně v indexu vidíme skupiny, podle kterých se dataset agregoval a do sloupců se daly všechny sloupce, na které bylo možné aplikovat naši agregační funkci (v tom případě na všechny zbylé).

Z čísel si můžeme všimnout, že u některých chybí údaje o datu narození nebo straně.

Než aplikujeme krok *apply*, můžeme si vybrat, na který ze sloupců tak učiníme, trochu si tím zpřehledníme výstup. Pokud si vybereme jen jeden sloupec, dostaneme Series.

In [9]:
legislators_historical_by_type.party.count()

type
rep    9925
sen    1822
Name: party, dtype: int64

**Úkol:** Rozděl data podle strany (`party`) a vypiš počet záznamů v každé skupině. Dokážeš výsledek setřídit podle velikosti skupin? 

In [10]:
# řešení
legislators_historical.groupby("party").last_name.count().sort_values(ascending=False).head(5)

party
Democrat      5005
Republican    4881
Whig           564
Federalist     343
Jackson        338
Name: last_name, dtype: int64

Možná jste si všimly/i, jaký je u výsledku index. Pokud ne, nevadí, určitě si všimnete teď. Zkusíme totiž vytvořit skupiny ne z jednoho sloupce, ale ze dvou. Pojďme si rozdělit zákonodárce podle států, a každou skupinu za jeden stát ještě podle pohlaví.

In [11]:
legislators_by_state_gender_counts = legislators_historical.groupby(["state", "gender"]).count()
legislators_by_state_gender_counts

last_name  first_name  birthday   type  party
state gender                                               
AK    F             NaN         NaN       NaN    NaN    NaN
      M            16.0        16.0      16.0   16.0   14.0
AL    F             3.0         3.0       3.0    3.0    3.0
      M           203.0       203.0     194.0  203.0  203.0
AR    F             5.0         5.0       5.0    5.0    5.0
...                 ...         ...       ...    ...    ...
WI    M           197.0       197.0     197.0  197.0  197.0
WV    F             1.0         1.0       1.0    1.0    1.0
      M           119.0       119.0     117.0  119.0  119.0
WY    F             2.0         2.0       2.0    2.0    2.0
      M            38.0        38.0      38.0   38.0   38.0

[116 rows x 5 columns]

Máme tedy skupiny, které jsou definované dvojicí hodnot stát a pohlaví (`state`, `gender`). A to je přesně důvod, proč existuje v Pandas [`MultiIndex`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.MultiIndex.html).

### Práce s MultiIndex

Vlastnosti MultiIndexu, vlastně takového víceúrovňového či vícerozměrného indexu, můžeme prozkoumat (kromě prostého zobrazení) pomocí několika užitečných atributů (properties).

In [12]:
# počet úrovní 
legislators_by_state_gender_counts.index.nlevels

2

In [13]:
# jména úrovní
legislators_by_state_gender_counts.index.names

FrozenList(['state', 'gender'])

In [14]:
# mohutnost (počet hodnot) jednotlivých úrovní
legislators_by_state_gender_counts.index.levshape

(58, 2)

In [15]:
# hodnoty v jednotlivých úrovních
legislators_by_state_gender_counts.index.levels

FrozenList([['AK', 'AL', 'AR', 'AS', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'DK', 'FL', 'GA', 'GU', 'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OL', 'OR', 'PA', 'PI', 'PR', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA', 'VI', 'VT', 'WA', 'WI', 'WV', 'WY'], ['F', 'M']])

Víme tedy, že náš (multi) index má dvě úrovně. Abychom dostali konkrétní řádek, musíme tím pádem zadat dvě hodnoty. K tomu nám poslouží `tuple` (pozor, musí to opravdu být `tuple` a ne `list`, tj. musíme použít kulaté a ne hranaté závorky).

In [16]:
legislators_by_state_gender_counts.loc[("WY", "F")]

last_name     2.0
first_name    2.0
birthday      2.0
type          2.0
party         2.0
Name: (WY, F), dtype: float64

Co kdybychom zadali jen polovinu indexu? Dostaneme celou skupinu, v našem případě celý stát.

In [17]:
legislators_by_state_gender_counts.loc["WY"]

,last_name,first_name,birthday,type,party
gender,,,,,
F,2.0,2.0,2.0,2.0,2.0
M,38.0,38.0,38.0,38.0,38.0


**Otázka:** Jaký je index výsledné tabulky?

Pokud bychom chtěli jedno pohlaví, můžeme indexu změnit pořadí.

In [18]:
swapped_index = legislators_by_state_gender_counts.index.swaplevel(0, 1)
legislators_by_gender_state_counts = legislators_by_state_gender_counts.set_index(swapped_index)
legislators_by_gender_state_counts

,,last_name,first_name,birthday,type,party
gender,state,,,,,
F,AK,NaN,NaN,NaN,NaN,NaN
M,AK,16.0,16.0,16.0,16.0,14.0
F,AL,3.0,3.0,3.0,3.0,3.0
M,AL,203.0,203.0,194.0,203.0,203.0
F,AR,5.0,5.0,5.0,5.0,5.0
...,...,...,...,...,...,...
M,WI,197.0,197.0,197.0,197.0,197.0
F,WV,1.0,1.0,1.0,1.0,1.0
M,WV,119.0,119.0,117.0,119.0,119.0


In [19]:
legislators_by_gender_state_counts.loc["F"].head()

,last_name,first_name,birthday,type,party
state,,,,,
AK,NaN,NaN,NaN,NaN,NaN
AL,3.0,3.0,3.0,3.0,3.0
AR,5.0,5.0,5.0,5.0,5.0
AS,NaN,NaN,NaN,NaN,NaN
AZ,3.0,3.0,3.0,3.0,3.0


Více o (pokročileší) práci s indexi a multiindexi najdeš v [dokumentaci](https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html).

Pokud bychom se chtěli multiindexu "zbavit", můžeme to udělat pomocí `.reset_index()`

In [20]:
legislators_by_state_gender_counts.reset_index().head(5)

,state,gender,last_name,first_name,birthday,type,party
0,AK,F,NaN,NaN,NaN,NaN,NaN
1,AK,M,16.0,16.0,16.0,16.0,14.0
2,AL,F,3.0,3.0,3.0,3.0,3.0
3,AL,M,203.0,203.0,194.0,203.0,203.0
4,AR,F,5.0,5.0,5.0,5.0,5.0


Anebo rovnou použít `groupby` s `as_index=False`.

In [21]:
legislators_historical.groupby(["state", "gender"], as_index=False).count().head(5)

,state,gender,last_name,first_name,birthday,type,party
0,AK,F,NaN,NaN,NaN,NaN,NaN
1,AK,M,16.0,16.0,16.0,16.0,14.0
2,AL,F,3.0,3.0,3.0,3.0,3.0
3,AL,M,203.0,203.0,194.0,203.0,203.0
4,AR,F,5.0,5.0,5.0,5.0,5.0


**Úkol:** TODO

## Co je ten `DataFrameGroupBy object`?

Na začátku jsme si udělali základní agregaci bez aplikace funkcí a dostali jsme jakýsi objekt. Je možné s ním něco dělat, aniž bychom agregovali? Ukazuje se, že ano.

In [11]:
legislators_historical.groupby('state')

Jedna klíčová funkce, kterou nám tento objekt nabízí, je iterace.

In [12]:
groups = legislators_historical.groupby('state')

next(iter(groups))

('AK',         last_name first_name   birthday gender type state        party
 6619       Waskey      Frank 1875-04-20      M  rep    AK     Democrat
 6647         Cale     Thomas 1848-09-17      M  rep    AK  Independent
 7442      Grigsby     George 1874-12-02      M  rep    AK          NaN
 7501       Sulzer    Charles 1879-02-24      M  rep    AK          NaN
 8039   Sutherland     Daniel 1869-04-17      M  rep    AK   Republican
 8236   Wickersham      James 1857-08-24      M  rep    AK   Republican
 8877       Dimond    Anthony 1881-11-30      M  rep    AK     Democrat
 9819     Gruening     Ernest 1887-02-06      M  sen    AK     Democrat
 9951       Rivers      Ralph 1903-05-23      M  rep    AK     Democrat
 9985     Bartlett     Edward 1904-04-20      M  sen    AK     Democrat
 10082     Pollock     Howard 1920-04-11      M  rep    AK   Republican
 10108      Begich   Nicholas 1932-04-06      M  rep    AK     Democrat
 10325      Gravel    Maurice 1930-05-13      M  sen    AK

Mírná odbočka zpět k Pythonu jako takovému. Máme-li nějaký zdroj tuplů:

In [13]:
data = [('Ondřej', 29), ('Královna Alžběta', 93), ('Maggie Simpson', 1)]

Pak můžeme v rámci iterace dostat buďto celý tuple, nebo můžeme rovnou *rozbalit* jednotlivé hodnoty do proměnných

In [15]:
for element in data:
    print(element)

('Ondřej', 29)
('Královna Alžběta', 93)
('Maggie Simpson', 1)


In [16]:
for name, age in data:
    print(f'{name}: {age} let')

Ondřej: 29 let
Královna Alžběta: 93 let
Maggie Simpson: 1 let


Zpět k naší agregaci - jak nám to pomůže? Podívejte zpět na to, co jsme dostali při iteraci nad `DataFrameGroupBy` objektem?

Ano, je to tuple.

In [17]:
groups = legislators_historical.groupby('state')

next(iter(groups))

('AK',         last_name first_name   birthday gender type state        party
 6619       Waskey      Frank 1875-04-20      M  rep    AK     Democrat
 6647         Cale     Thomas 1848-09-17      M  rep    AK  Independent
 7442      Grigsby     George 1874-12-02      M  rep    AK          NaN
 7501       Sulzer    Charles 1879-02-24      M  rep    AK          NaN
 8039   Sutherland     Daniel 1869-04-17      M  rep    AK   Republican
 8236   Wickersham      James 1857-08-24      M  rep    AK   Republican
 8877       Dimond    Anthony 1881-11-30      M  rep    AK     Democrat
 9819     Gruening     Ernest 1887-02-06      M  sen    AK     Democrat
 9951       Rivers      Ralph 1903-05-23      M  rep    AK     Democrat
 9985     Bartlett     Edward 1904-04-20      M  sen    AK     Democrat
 10082     Pollock     Howard 1920-04-11      M  rep    AK   Republican
 10108      Begich   Nicholas 1932-04-06      M  rep    AK     Democrat
 10325      Gravel    Maurice 1930-05-13      M  sen    AK

Můžeme-li tedy dostat tuple při každé iteraci, můžeme jej rozbalit. Konkrétně to je dvoupoložkový index, kde první element je hodnota, nad kterou je agregováno, a druhý element je celý DataFrame, který této hodnotě odpovídá!

In [18]:
for index, df in groups:
    break

In [19]:
index

'AK'

In [20]:
df

,last_name,first_name,birthday,gender,type,state,party
6619,Waskey,Frank,1875-04-20,M,rep,AK,Democrat
6647,Cale,Thomas,1848-09-17,M,rep,AK,Independent
7442,Grigsby,George,1874-12-02,M,rep,AK,NaN
7501,Sulzer,Charles,1879-02-24,M,rep,AK,NaN
8039,Sutherland,Daniel,1869-04-17,M,rep,AK,Republican
8236,Wickersham,James,1857-08-24,M,rep,AK,Republican
8877,Dimond,Anthony,1881-11-30,M,rep,AK,Democrat
9819,Gruening,Ernest,1887-02-06,M,sen,AK,Democrat
9951,Rivers,Ralph,1903-05-23,M,rep,AK,Democrat
9985,Bartlett,Edward,1904-04-20,M,sen,AK,Democrat


K čemu nám to může být? Při agregaci se data drasticky zjednodušují a nemusíme si vždy být jisti, že naše agregace jsou napsané správně. Pomocí iterace nad skupinami si můžeme zobrazit všechna data před agregací.

In [21]:
import os
target_directory = 'po_statech'
os.makedirs(target_directory, exist_ok=True)

In [22]:
for state, df in legislators_historical.groupby('state'):
    df.to_csv(os.path.join(target_directory, state + '.csv'))

## Další agregační funkce
nejen count

In [23]:
legislators_historical.groupby('state').agg({'birthday': ['min', 'max']}).head()

birthday           
             min        max
state                      
AK    1848-09-17 1962-03-30
AL    1780-01-30 1967-10-09
AR    1788-08-25 1968-08-21
AS    1937-03-13 1943-08-15
AZ    1816-01-24 1976-11-03

co je moznyho s agg?

## Agregace nad nesloupci

Doteď jsme vždy agregovali nad něčím, co jsme měli v DataFramu. Je ale možné agregovat nad daty, které tam vůbec nemáme, alespoň ne explicitně.

Kromě názvu sloupce můžeme do `groupby` vložit nějakou Series (!), která má stejný tvar jako naše sloupce a pandas podle toho bude umět agregovat. Jaká taková Series se nabízí? Nejlépe transformace nějakého existujícího sloupce.

Na začátku jsme si ukazovali, že občas chybí nějaká data, konkrétně birthday a party.

In [24]:
legislators_historical.party.isnull()

0        False
1         True
2         True
3         True
4         True
         ...  
11976    False
11977    False
11978    False
11979    False
11980    False
Name: party, Length: 11981, dtype: bool

Series má metodu `isnull`, která nám vrátí True/False hodnoty podle toho, jestli daná hodnota je platná nebo ne (chybějící, NULL v SQL).

Tuto Series můžeme použít pro agregaci.

In [25]:
legislators_historical.groupby([
    legislators_historical.state,
    legislators_historical.party.isnull()])[['last_name']].count()

last_name
state party           
AK    False       14.0
      True         2.0
AL    False      206.0
      True         NaN
AR    False      115.0
...                ...
WI    True         NaN
WV    False      120.0
      True         NaN
WY    False       40.0
      True         NaN

[116 rows x 1 columns]

Ukázali jsme si agregaci na jednoduché metodě `.isnull`, ale na čem dalším to jde udělat?

### .str a .dt metody

https://pandas.pydata.org/pandas-docs/stable/user_guide/text.html#string-methods

groupby str.lower() to dedup

In [37]:
# name_length = legislators_historical.first_name.str.len()

In [43]:
birth_years = legislators_historical.birthday.dt.year

birth_years#.value_counts()

0        1745.0
1        1742.0
2        1743.0
3        1730.0
4        1739.0
          ...  
11976    1950.0
11977    1951.0
11978    1987.0
11979    1944.0
11980    1976.0
Name: birthday, Length: 11981, dtype: float64

In [46]:
legislators_historical.groupby(birth_years)

### Binning

cut, qcut, value_counts s prepinacem

## Pivoting - TODO

> pivot (third-person singular simple present pivots, present participle pivoting, simple past and past participle pivoted)
 **To turn on an exact spot.**
 
> A pivot table is a table of statistics that summarizes the data of a more extensive table ...
> Although pivot table is a generic term, Microsoft Corporation trademarked PivotTable in the United States in 1994.

Our pivoting task: Get a table with numbers of titles per year (as row) and type (as column).

One approach is to use `groupby`, `count` aggregation and `unstack`.

In [37]:
grouped_by_year_and_type = imdb_data.groupby(["startYear", "titleType"])

NameError: name 'imdb_data' is not defined

In [ ]:
pivoted = grouped_by_year_and_type.numVotes.count().unstack()
pivoted.tail()

There's a shortcut though, see if you we can use it.

**Exercise:** Create the `pivoted` table using `pivot_table`:

In [ ]:
%exercise

pivot_table = imdb_data.pivot_table(values=___, index=___, columns=___, aggfunc=___)

# display - do not edit
pivot_table.tail()

In [ ]:
%validate

pd.testing.assert_frame_equal(pivoted, pivot_table)

We can now use this to plot a kind of a histogram with colour for title types.

In [ ]:
with sns.color_palette("Paired"):
    fig, ax = plt.subplots(figsize=(16, 6))
    pivoted.loc[1990:].plot.bar(stacked=True, ax=ax)

## Final mini-project - creative, unbounded, free-style

**Tohle je z Pycon workshopu - kdyby se to třeba hodilo ...**

Here are some ideas of what you can do with the data.

* Create 5-star rating based on quantiles using `quantile` and `cut` or `qcut`.
* Group by studio / decade / rating
* Compare simple (arithmetic) mean `averageRating` in each group with `averageRating` average weighted by `numVotes` ($ \frac{\sum \rm{averageRating} \times \rm{numVotes}} {\sum \rm{numVotes}} $). Use `apply` and the `wavg` function from https://pbpython.com/weighted-average.html. This function is quite time and memory consuming and thus not ideal for large data sets. You can try to implement weighted average using standard `mean`. Check the performance with the `%timeit` magic.
* Use the 5-star rating for `hue` in an interesting seaborn plot (see https://seaborn.pydata.org/tutorial/relational.html)
* Use `sns.catplot` to visualize the distrubution of incomes in each 5-star rating group. 

A couple more ideas can be found in https://github.com/brandon-rhodes/pycon-pandas-tutorial

After you have solved all of those, come up with your own quests - we may still be around and help you :-D